In [6]:
import time
from uk_covid19 import Cov19API
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [29]:
# our API access function. This will be called by the button when it is clicked
def access_api():  
    filters1 = [
        "areaType=overview"
    ]

    structure1 = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate",
    }
    
    filters2 = [
        'areaType=nation',
        'areaName=England'
    ]

    structure2 = {
        "date": "date",
        "hospital": "newAdmissions",
        "ventilator": "covidOccupiedMVBeds"
    }

    
    filters3 = [
        'areaType=region',
        'areaName=London'
    ]

    structure3 = {
        "date": "date",
        "deaths": "newDeaths28DaysByDeathDate",
        "cases": "newCasesBySpecimenDate"
    }
    
    
    api_overview = Cov19API(filters=filters1, structure=structure1)
    overview = api_overview.get_json()
    
    time.sleep(1)
    
    api_ventilator = Cov19API(filters=filters2, structure=structure2)
    ventilator = api_ventilator.get_json()
    
    time.sleep(1)
    
    api_london = Cov19API(filters=filters3, structure=structure3)
    london_cases_deaths = api_london.get_json()

    with open("overview.json", "wt") as OUTF:
        json.dump(overview, OUTF)

    with open("ventilator.json", "wt") as OUTF:
        json.dump(ventilator, OUTF)

    with open("london_cases_deaths.json", "wt") as OUTF:
        json.dump(london_cases_deaths, OUTF)



    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api())

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)


def parse_date(date_string):
    ''' Convert a date string into a pandas datetime object '''
    return pd.to_datetime(date_string, format="%Y-%m-%d")


def create_df1(file):
    '''creates a pandas dataframe from a "json" dictionary from uk-covid19 API software development kit'''
    with open(file, "rt") as INFILE:
        data = json.load(INFILE)
    data_list =data['data']
    dates = [dictionary['date'] for dictionary in data_list]
    dates.sort()
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    
    index = pd.date_range(startdate, enddate, freq = 'D')
    df = pd.DataFrame(index=index, columns= ['cases', 'hospital','deaths'])
    
    for entry in data_list:
        date = parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                df.loc[date, column] = value
                
    df.fillna(0.0, inplace = True)
    return df


def create_df2(file):
    '''creates a pandas dataframe from a "json" dictionary from uk-covid19 API software development kit'''
    with open(file, "rt") as INFILE:
        data = json.load(INFILE)
    data_list =data['data']
    dates = [dictionary['date'] for dictionary in data_list]
    dates.sort()
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    
    index = pd.date_range(startdate, enddate, freq = 'D')
    df = pd.DataFrame(index=index, columns= ['hospital','ventilator'])
    
    for entry in data_list:
        date = parse_date(entry['date'])
        for column in ['hospital','ventilator']:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                df.loc[date, column] = value
                
    df.fillna(0.0, inplace = True)
    return df


def create_df3(file):
    '''creates a pandas dataframe from a "json" dictionary from uk-covid19 API software development kit'''
    with open(file, "rt") as INFILE:
        data = json.load(INFILE)
    data_list =data['data']
    dates = [dictionary['date'] for dictionary in data_list]
    dates.sort()
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    
    index = pd.date_range(startdate, enddate, freq = 'D')
    df = pd.DataFrame(index=index, columns= ['deaths','cases'])
    
    for entry in data_list:
        date = parse_date(entry['date'])
        for column in ['deaths','cases']:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                df.loc[date, column] = value
                
    df.fillna(0.0, inplace = True)
    return df



I'm downloading data from the API...
API call finished succesfully


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [30]:
df1 = create_df1("overview.json")
df2 = create_df2("ventilator.json")
df3 = create_df3("london_cases_deaths.json")

ok


In [31]:
series1 = wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale1 = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls1 = wdg.HBox([series1, scale1])


series2 = wdg.SelectMultiple(
    options=['hospital', 'ventilator'],
    value=['hospital', 'ventilator'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale2 = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=True
)

controls2 = wdg.HBox([series2, scale2])



def overview_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        df1[list(gcols)].plot(logy=logscale)
        print("(CTRL-Click to select more than one category)")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_overview = wdg.interactive_output(overview_graph, {'gcols': series1, 'gscale': scale1})

display(controls1, output_overview)


def ventilator_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        df2[list(gcols)].plot(logy=logscale)
        print("(CTRL-Click to select more than one category)")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_ventilator = wdg.interactive_output(ventilator_graph, {'gcols': series2, 'gscale': scale2})

display(controls2, output_ventilator)

    
    
    



Output()

Output()